In [1]:
import sys
import os
from urllib.request import urlopen

sys.path.append("/home/qb/Desktop/GPs")


import torch
import numpy as np
import pandas as pd
import tqdm

from data import GaussianPreprocessor, SequenceDataset
from models import LSTMFeatureExtractor


In [2]:
def download_data():
    if not os.path.exists("eeg.dat"):
        url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00264/EEG%20Eye%20State.arff"
        with open("eeg.dat", "wb") as f:
            f.write(urlopen(url).read())

In [3]:
if ('CI' in os.environ):  # this is for running the notebook in our testing framework
    train_set = torch.utils.data.TensorDataset(torch.randn(8, 3, 32, 32), torch.rand(8).round().long())
    test_set = torch.utils.data.TensorDataset(torch.randn(4, 3, 32, 32), torch.rand(4).round().long())
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=2, shuffle=False)
    num_classes = 2
else:
    download_data()
    T_forecast = 349
    data = np.loadtxt("eeg.dat", delimiter=",", skiprows=19)
    print("[raw data shape] {}".format(data.shape))
    data = torch.tensor(data[::20, :-1]).float()
    print("[data shape after thinning] {}".format(data.shape))
    
    T, obs_dim = data.shape
    T_train = T - T_forecast

    # standardize data
    data_mean = data[0:T_train, :].mean(0)
    data -= data_mean
    data_std = data[0:T_train, :].std(0)
    data /= data_std

    torch.manual_seed(1080)

[raw data shape] (14980, 15)
[data shape after thinning] torch.Size([749, 14])


In [14]:
dts = (1 + torch.arange(10)).double()
print(dts)

tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.], dtype=torch.float64)


In [4]:
# batch_size=10
# train_T = 5
# forecast_T = batch_size - train_T

# num_batches = data.shape[0] - batch_size

# num_epochs = 1000

In [5]:
# class LSTMEncoderDecoder(torch.nn.Module):
#     def __init__(self):
#         super(LSTMEncoderDecoder, self).__init__()
#         self.lstm_encoder = LSTMFeatureExtractor(14, 64, 0, num_layers=2, bidirectional=True, dropout=0.2)
#         self.lstm_decoder = LSTMFeatureExtractor(2 * 64, 64, 14, num_layers=2, bidirectional=False, dropout=0.2)
        
#     def forward(self, x):
#         x, (h, c) = self.lstm_encoder(x)
#         return self.lstm_decoder(x)
    
# model = LSTMEncoderDecoder().cuda()

# print(model)

LSTMEncoderDecoder(
  (lstm_encoder): LSTMFeatureExtractor(
    (rnn): LSTM(14, 64, num_layers=2, dropout=0.2, bidirectional=True)
  )
  (lstm_decoder): LSTMFeatureExtractor(
    (rnn): LSTM(128, 64, proj_size=14, num_layers=2, dropout=0.2)
  )
)


In [6]:
# set up optimizer
adam = torch.optim.Adam(
    model.parameters(),
    lr=0.1,
    )
    # we decay the learning rate over the course of training
gamma = (0.001 / 0.01) ** (
        1.0 / num_epochs
    )
    
scheduler = torch.optim.lr_scheduler.ExponentialLR(adam, gamma=gamma)

loss_fn = torch.nn.MSELoss()

In [7]:
# epochs_iter = tqdm.notebook.tqdm(range(num_epochs), desc="Epoch")
# for i in epochs_iter:
#     # Within each iteration, we will go over each minibatch of data
    
#     adam.zero_grad()
    
#     start_index = 0
    
#     losses = []
    
#     for i in range(num_batches):
# #         print(i)
#         batch = data[start_index: start_index + train_T, :].cuda()
#         target = data[start_index + train_T: start_index + batch_size, :].cuda()
#         output, _ = model(batch)
#         losses.append(loss_fn(output, target))
#         losses[-1].backward()
#         start_index += 1
    
#     total_loss = 0
#     for loss in losses:
#         total_loss += loss.item()
    
#     epochs_iter.set_postfix(loss=total_loss)
# #     writer.add_scalar('Loss/train', total_loss, i)
#     adam.step()

Epoch:   0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:


# test_x = data[100: 100 + train_T, :]
# test_y = data[100 + train_T: 100 + batch_size, :]


# pred, _ = model(test_x.cuda())

# print(pred)
# print(test_y)

tensor([[-0.2235, -0.0534, -0.0919, -0.2298, -0.0076, -0.1813, -0.5084, -0.0755,
         -0.1404, -0.2393, -0.0631, -0.3437, -0.1596, -0.1141],
        [-0.2046, -0.0358, -0.0195, -0.2164,  0.1551, -0.1185, -0.5436, -0.0204,
         -0.1387, -0.2142, -0.0543, -0.3537, -0.1565, -0.0977],
        [-0.1383,  0.0428,  0.0341, -0.0823,  0.2592,  0.0322, -0.4231,  0.0606,
         -0.0928, -0.1328, -0.0286, -0.3320, -0.0992, -0.0806],
        [-0.2175, -0.0401, -0.0439, -0.2083,  0.0901, -0.0782, -0.5182,  0.0093,
         -0.0894, -0.1775, -0.0429, -0.3433, -0.1478, -0.0808],
        [-0.2256, -0.0243, -0.0279, -0.2059,  0.1067, -0.1033, -0.5236,  0.0020,
         -0.0871, -0.2036, -0.0162, -0.3664, -0.1454, -0.1068]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([[ 4.2850e-01,  4.5847e-01,  3.2603e-01,  3.8882e-01, -4.7573e-01,
         -6.6099e-01, -8.7429e-02, -1.0739e+00, -3.5041e-01,  1.5334e-01,
         -1.6457e+00,  3.3230e-01,  2.2536e-01,  2.9109e-01],
        [-2.